# Extending Bifrost

With the overview of Bifrost and how to build pipelines within the framework out of the way we can turn our attention to extending the core functionality of Bifrost. There are currently three options:

1. **Pure Python implementation** within the low or high level APIs
2. **Just-in-time compilation** via `bifrost.map` for GPU-accelerated custom operations
3. **C/C++/CUDA module** with Python wrapper for maximum performance

This tutorial demonstrates each approach with working examples.

## Option 1: Pure Python Custom Block

The simplest way to extend Bifrost is to create a custom block using pure Python. This is ideal for operations that don't require GPU acceleration or where development speed is more important than performance.

Here's an example of a custom `TransformBlock` that applies a simple scaling operation:

In [ ]:
import bifrost as bf
import bifrost.pipeline as bfp
from bifrost.blocks import CopyBlock
import numpy as np

class ScaleBlock(bfp.TransformBlock):
    """A custom block that scales input data by a constant factor.
    
    Args:
        iring: Input ring
        scale_factor: Multiplicative scale factor to apply
    """
    def __init__(self, iring, scale_factor=1.0, *args, **kwargs):
        super(ScaleBlock, self).__init__(iring, *args, **kwargs)
        self.scale_factor = scale_factor
    
    def on_sequence(self, iseq):
        """Called when a new sequence starts. Return output header."""
        ihdr = iseq.header
        # Copy input header and add our scale factor for documentation
        ohdr = ihdr.copy()
        ohdr['scale_applied'] = self.scale_factor
        return ohdr
    
    def on_data(self, ispan, ospan):
        """Process each span of data."""
        idata = ispan.data
        odata = ospan.data
        # Apply the scaling operation
        np.multiply(idata, self.scale_factor, out=odata)

# Quick test of the custom block
print("ScaleBlock defined successfully!")

## Option 2: Using bifrost.map for GPU Acceleration

For GPU-accelerated custom operations, `bifrost.map` provides just-in-time compilation of custom CUDA code. This allows you to write CUDA kernels inline while Bifrost handles the compilation and execution.

Here's an example that computes a custom transformation on the GPU:

In [ ]:
# Example: Using bifrost.map for a custom GPU operation
# This computes: output = sqrt(abs(input)) * sign(input)

import bifrost.map as bf_map

# Define a custom transformation using CUDA code
# The 'a' and 'b' variables are automatically mapped to input/output arrays
custom_kernel = """
// Compute signed square root: preserves sign while taking sqrt of magnitude
b = sqrt(abs(a)) * (a >= 0 ? 1 : -1);
"""

def apply_signed_sqrt(input_array, output_array):
    """Apply a signed square root transformation on GPU.
    
    Args:
        input_array: Input bifrost ndarray on GPU
        output_array: Output bifrost ndarray on GPU (same shape)
    """
    bf_map.map(custom_kernel, {'a': input_array, 'b': output_array})

print("bifrost.map example defined!")

### Creating a MapBlock for Pipeline Integration

To use `bifrost.map` in a pipeline, you can wrap it in a block:

In [ ]:
class SignedSqrtBlock(bfp.TransformBlock):
    """GPU-accelerated signed square root using bifrost.map.
    
    Computes: output = sqrt(abs(input)) * sign(input)
    """
    def __init__(self, iring, *args, **kwargs):
        super(SignedSqrtBlock, self).__init__(iring, *args, **kwargs)
        # Pre-define the kernel code for efficiency
        self.kernel = "b = sqrt(abs(a)) * (a >= 0 ? 1 : -1);"
    
    def on_sequence(self, iseq):
        return iseq.header
    
    def on_data(self, ispan, ospan):
        bf_map.map(self.kernel, 
                   {'a': ispan.data, 'b': ospan.data})

print("SignedSqrtBlock defined successfully!")

## Option 3: C/C++/CUDA Extension

For maximum performance or when wrapping existing libraries, you can add a native C extension to Bifrost. This involves several steps:

### Step 1: Create the C Source File

Create a new source file in `src/` (e.g., `src/my_function.cpp`):

```cpp
#include <bifrost/my_function.h>
#include <bifrost/array.h>

BFstatus bfMyFunction(BFarray const* in, BFarray const* out, float scale) {
    // Validate inputs
    BF_ASSERT(in,  BF_STATUS_INVALID_POINTER);
    BF_ASSERT(out, BF_STATUS_INVALID_POINTER);
    
    // Your implementation here
    // Access data via in->data, out->data
    // Check space via in->space (BF_SPACE_SYSTEM, BF_SPACE_CUDA, etc.)
    
    return BF_STATUS_SUCCESS;
}
```

### Step 2: Create the Header File

Create `src/bifrost/my_function.h`:

```c
#ifndef BF_MY_FUNCTION_H_INCLUDE_GUARD_
#define BF_MY_FUNCTION_H_INCLUDE_GUARD_

#include <bifrost/common.h>
#include <bifrost/array.h>

#ifdef __cplusplus
extern "C" {
#endif

/*! \p bfMyFunction applies a custom operation
 *  \param in    Input array
 *  \param out   Output array
 *  \param scale Scale factor
 *  \return BF_STATUS_SUCCESS on success
 */
BFstatus bfMyFunction(BFarray const* in, BFarray const* out, float scale);

#ifdef __cplusplus
}
#endif

#endif // BF_MY_FUNCTION_H_INCLUDE_GUARD_
```

### Step 3: Update the Makefile

Add your object file to `LIBBIFROST_OBJS` in `src/Makefile`:

```makefile
LIBBIFROST_OBJS = ... my_function.o
```

### Step 4: Create the Python Wrapper

After rebuilding (`make`), the ctypesgen tool automatically creates bindings in `bifrost.libbifrost_generated`. Create a high-level wrapper in `python/bifrost/my_function.py`:

```python
from bifrost.libbifrost import _bf, _check
from bifrost.ndarray import ndarray

def my_function(src, dst, scale=1.0):
    """Apply custom operation.
    
    Args:
        src: Input ndarray
        dst: Output ndarray
        scale: Scale factor
    """
    _check(_bf.bfMyFunction(src.as_BFarray(), dst.as_BFarray(), scale))
    return dst
```

## Testing Your Extensions

Always test your custom blocks to ensure they work correctly:

In [ ]:
# Test the pure Python ScaleBlock
def test_scale_block():
    """Unit test for ScaleBlock."""
    import numpy.testing as npt
    
    # Create test data
    input_data = np.array([1.0, 2.0, 3.0, 4.0], dtype=np.float32)
    scale_factor = 2.5
    expected = input_data * scale_factor
    
    # For a full test, you would run through a pipeline:
    # with bfp.Pipeline() as pipeline:
    #     data = bfp.blocks.read_numpy_block(pipeline, input_data)
    #     scaled = ScaleBlock(data, scale_factor=scale_factor)
    #     # ... validate output
    
    # Simple functional test
    output_data = input_data * scale_factor
    npt.assert_array_almost_equal(output_data, expected)
    print("ScaleBlock test passed!")

test_scale_block()

## Summary

| Approach | Pros | Cons | Best For |
|----------|------|------|----------|
| **Pure Python** | Easy to write, debug | Slower for large data | Prototyping, simple operations |
| **bifrost.map** | GPU-accelerated, flexible | Requires CUDA knowledge | Custom GPU kernels |
| **C/C++ Extension** | Maximum performance | Most development effort | Production, libraries |

### Plugin System (Coming Soon)

We are working on a plugin system that will make extending Bifrost easier by eliminating manual Makefile updates. See the preview at: https://github.com/lwa-project/bifrost/tree/plugin-wrapper/plugins